In [0]:
%run /Users/rishi@bodo.ai/setup/setup_nb_px

import java.io.File
import java.nio.file.{Files, Path, Paths, StandardCopyOption}
import java.nio.file.attribute.PosixFilePermissions
import java.text.SimpleDateFormat
import java.util.{Date, Properties}
import scala.sys.process._
import org.apache.spark.SparkContext

publicKey: String =
"ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCjAnYtDQf+sEHJSKn3PNU0A0o8auMuKpcKo9lvKbx0ApAPxJ0DAVjczOhZ/is3mJeDiQdH0CdU8tggb+DYvNw4BXlSgbv7v0Lx0SUvxFOFRycCLDAo2Jk9tQEChwW0BtgD1Xq5Jld1oJ8+Y10YwfAoKFGFJPD+zS4ONkpc3c9XtbIY1a76qKJK/FR+4Tz9EWqB8wckE5tCU+dpa3DhQfyt3QSHk4PDlgcTWBCqXObKmLOaxu6309NkRMgR705X06qIx3gTqBnd7XE6tl07E629S7wiDQrUflfvnDHUAxTQwN4kRmJmIh+AWTEq/LXctVatHaf559BlzDodVNTpAQs1 root@1014-132551-ywrn3zez-10-44-235-163
"
addAuthorizedPublicKey: (key: String)Unit
numExecutors: Int = 3

Executors = Set(10.44.233.230, 10.44.235.163, 10.44.227.112)
Workers = List(10.44.233.230, 10.44.235.163, 10.44.227.112)
workers: List[String] = List(10.44.233.230, 10.44.235.163, 10.44.227.112)
writeFile: (path: String, contents: String, append: Boolean)Unit
addHostfile: (key: String)Unit

createIpyprofile: (logStdout: Boolean)String
numExecutors: Int = 3

createSSHfolder: (logStdout: Boolean)String
numExecutors: Int = 3

In [0]:
%fs
ls /mnt/rjml2/temp/demand-forecasting/train.csv

path,name,size
dbfs:/mnt/rjml2/temp/demand-forecasting/train.csv,train.csv,17333449


publicKey: String =
"ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/m99pFCWf0sW8MLqHD/37QoGPmyR+LLD3j+AEvr6ymsCgdyez10mN1WVcVZ43Ouf9sbVVkgYVpr8g9Pb1LqGJzB5nQ7Yh3x3lFOzwX3L9AgZeRd2RWyJOfRs7mGlhwwyWTZK3rApmF7R6cAwSErG1lfwnsboYeBi+UQtGs+V0kqKlIhAQj4mpiP3XElRad1shAfCsVYbmBAeEjfYOaaO1ruMQQuhf8zAYQk+je95Mzys5vPJceoeP2j4+SX8gMkIXK3/4zwBXBOK1u7AWmc3PfzAzXn+AspsEfz2D0RqSEGYeS8Gn0nezE65lSmoV6V6GLqoP+HG2OsW4vS6oBNaP root@1014-132551-ywrn3zez-10-44-235-163
"
addAuthorizedPublicKey: (key: String)Unit
numExecutors: Int = 3

startSSHserver: (logStdout: Boolean)String
numExecutors: Int = 3

ssh'ing onto host - 10.44.233.230
Pseudo-terminal will not be allocated because stdin is not a terminal.
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 5.4.0-1055-aws x86_64)

 * Documentation: https://help.ubuntu.com
 * Management: https://landscape.canonical.com
 * Support: https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.
SUCCESS: on host: 10.44.233.230
ssh'ing onto host - 10.44.235.163
Pseudo-terminal will not be allocated because stdin is not a terminal.
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 5.4.0-1055-aws x86_64)

 * Documentation: https://help.ubuntu.com
 * Management: https://landscape.canonical.com
 * Support: https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.
SUCCESS: on host: 10.44.235.163
ssh'ing onto host - 10.44.227.112
Pseudo-terminal will not be allocated because stdin is not a terminal.
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 5.4.0-1055-aws x86_64)

 * Documentation: https://help.ubuntu.com
 * Management: https://landscape.canonical.com
 * Support: https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.
SUCCESS: on host: 10.44.227.112
ssh: (host: String, logStdout: Boolean)String

ubuntu
Python 3.8.8
[1] 1014-132551-ywrn3zez-10-44-235-163
[0] 1014-132551-ywrn3zez-10-44-233-230
[2] 1014-132551-ywrn3zez-10-44-227-112

Out[5]: '2021.8'

Out[6]: '7.1.0'

Using existing profile dir: '/root/.ipython/profile_mpi'

Starting 12 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
INFO:ipyparallel.cluster.cluster.1634229707-abf9:Starting 12 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
 0%| | 0/12 [00:00<?, ?engine/s] 8%|▊ | 1/12 [00:11<02:03, 11.27s/engine] 75%|███████▌ | 9/12 [00:14<00:03, 1.25s/engine] 83%|████████▎ | 10/12 [00:17<00:02, 1.50s/engine]100%|██████████| 12/12 [00:17<00:00, 1.44s/engine]

In [0]:
%%px

import numpy as np
import pandas as pd
import time
import datetime
import bodo
import os
import warnings
import matplotlib.pyplot as plt
%matplotlib inline

from prophet import Prophet
import logging
 
# disable informational messages from fbprophet
logging.getLogger('py4j').setLevel(logging.ERROR)

# Turn off pystan warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)

# Turn off fbprophet stdout logger
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('pystan').setLevel(logging.ERROR)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:12<?, ?tasks/s]%px: 0%| | 0/12 [00:12<?, ?tasks/s]%px: 0%| | 0/12 [00:12<?, ?tasks/s]%px: 0%| | 0/12 [00:12<?, ?tasks/s]%px: 0%| | 0/12 [00:12<?, ?tasks/s]%px: 0%| | 0/12 [00:12<?, ?tasks/s]%px: 33%|███▎ | 4/12 [00:12<00:00, 35.43tasks/s]%px: 67%|██████▋ | 8/12 [00:14<00:01, 3.92tasks/s]%px: 75%|███████▌ | 9/12 [00:25<00:06, 2.07s/tasks]%px: 83%|████████▎ | 10/12 [00:25<00:03, 1.75s/tasks]%px: 92%|█████████▏| 11/12 [00:28<00:02, 2.06s/tasks]%px: 100%|██████████| 12/12 [00:32<00:00, 2.39s/tasks]%px: 100%|██████████| 12/12 [00:32<00:00, 2.68s/tasks]

[stdout:0] 
Hello World from rank 0 on host 1014-132551-ywrn3zez-10-44-233-230. total ranks=12
[stdout:1] 
Hello World from rank 1 on host 1014-132551-ywrn3zez-10-44-235-163. total ranks=12
[stdout:9] 
Hello World from rank 9 on host 1014-132551-ywrn3zez-10-44-233-230. total ranks=12
[stdout:7] 
Hello World from rank 7 on host 1014-132551-ywrn3zez-10-44-235-163. total ranks=12
[stdout:4] 
Hello World from rank 4 on host 1014-132551-ywrn3zez-10-44-235-163. total ranks=12
[stdout:3] 
Hello World from rank 3 on host 1014-132551-ywrn3zez-10-44-233-230. total ranks=12
[stdout:6] 
Hello World from rank 6 on host 1014-132551-ywrn3zez-10-44-233-230. total ranks=12
[stdout:10] 
Hello World from rank 10 on host 1014-132551-ywrn3zez-10-44-235-163. total ranks=12
[stdout:8] 
Hello World from rank 8 on host 1014-132551-ywrn3zez-10-44-227-112. total ranks=12
[stdout:11] 
Hello World from rank 11 on host 1014-132551-ywrn3zez-10-44-227-112. total ranks=12
[stdout:5] 
Hello World from rank 5 on host 1014-132551-ywrn3zez-10-44-227-112. total ranks=12
[stdout:2] 
Hello World from rank 2 on host 1014-132551-ywrn3zez-10-44-227-112. total ranks=12

In [0]:
%%px

@bodo.jit()
def load_data():
    dtype = {'store': np.dtype('int64'), 'item': np.dtype('int64'), 
             'sales': np.dtype('int64')}
    train = pd.read_csv('/dbfs/mnt/rjml2/temp/demand-forecasting/train.csv', dtype = dtype, parse_dates=['date'])
    # train = pd.read_csv('demand-forecasting/train.csv', parse_dates=['date'])
    #print(train.head(3))
    return train

train = load_data()
if bodo.get_rank() == 0:
    display(train.head(2))
    print(train.info())

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px:

In [0]:
%%px

@bodo.jit()
def yearly_trend(train):
    train['year'] = train['date'].dt.year
    Yearly_trend = train.groupby('year', as_index = False)['sales'].sum()
    Yearly_trend = Yearly_trend.sort_values(['year'], ascending = True)
    print(bodo.allgatherv(Yearly_trend))
    return bodo.allgatherv(Yearly_trend), train

Yearly_trend, train = yearly_trend(train)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px:

In [0]:
%%px

#BE-1419
#@bodo.jit()
#def yearly_trend_plot(Yearly_trend):
if bodo.get_rank() == 0:
  g = Yearly_trend.plot(x = 'year', y = 'sales')
  #plt.show()
  display(g)
#yearly_trend_plot(Yearly_trend)

[output:0]
<AxesSubplot:xlabel='year'>
[output:0]
<Figure size 432x288 with 1 Axes>

In [0]:
%%px

@bodo.jit()
def monthly_trend(train):
    train['year-month'] = train['date'].dt.strftime('%Y-%m')
    Monthly_trend = train.groupby(['year-month'], as_index = False)['sales'].sum()
    Monthly_trend = Monthly_trend.sort_values(['year-month'], ascending = (True))
    print(bodo.allgatherv(Monthly_trend))
    return bodo.allgatherv(Monthly_trend), train

Monthly_trend, train = monthly_trend(train)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px:

In [0]:
%%px

#BE-1419
#@bodo.jit()
#def monthly_trend_plot(Monthly_trend):
if bodo.get_rank() == 0:
    g = Monthly_trend.plot(x = 'year-month', y = 'sales')
    display(g)
    #plt.show()
    
#monthly_trend_plot(Monthly_trend)

[output:0]
<AxesSubplot:xlabel='year-month'>
[output:0]
<Figure size 432x288 with 1 Axes>

In [0]:
%%px

@bodo.jit()
def weekly_trend(train):
    train['week-day'] = train['date'].dt.dayofweek
    Weekly_trend = train.groupby(['year','week-day'], as_index = False)['sales'].sum()
    Weekly_trend = Weekly_trend.sort_values(['year','week-day'], ascending = (True, True))
    print(bodo.allgatherv(Weekly_trend))
    return Weekly_trend, train

Weekly_trend = weekly_trend(train)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s][stdout:0] 
 year week-day sales
21 2013 0 898024
16 2013 1 1060838
24 2013 2 1047808
20 2013 3 1118166
5 2013 4 1199697
25 2013 5 1275711
34 2013 6 1340999
1 2014 0 1029183
9 2014 1 1205326
14 2014 2 1219502
0 2014 3 1289906
6 2014 4 1369927
26 2014 5 1466786
17 2014 6 1554852
2 2015 0 1080971
31 2015 1 1252678
7 2015 2 1257066
22 2015 3 1365475
27 2015 4 1437323
28 2015 5 1520251
18 2015 6 1623123
29 2016 0 1165607
19 2016 1 1364934
12 2016 2 1368489
15 2016 3 1455690
3 2016 4 1580478
11 2016 5 1671927
32 2016 6 1750035
30 2017 0 1212068
10 2017 1 1409705
13 2017 2 1419225
8 2017 3 1520643
23 2017 4 1610596
4 2017 5 1720807
33 2017 6 1840696
%px: 100%|██████████| 12/12 [00:08<00:00, 1.50tasks/s]

In [0]:
%%px

@bodo.jit()
def single_store(train):
    history_pd = train.loc[(train['store']== 1) & (train['item']== 1)]
    history_pd = history_pd['date', 'sales'].rename(columns = {'date' : 'ds', 'sales': 'y'})
    history_pd = history_pd.dropna()
    history_pd = history_pd.sort_values('ds', ascending = True)
    print(history_pd.head())
    return history_pd
history_pd = single_store(train)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px:

In [0]:
%%px

sample_df = pd.DataFrame({"ds": ["2020-01-05", "2020-01-06"], "yhat": ["5623", "6354"]})

from numba.core import types
types.mydataframe = bodo.typeof(sample_df)

In [0]:
%%px


@bodo.jit()
def train_prophet(history_pd):
    
    with bodo.objmode(forecast_pd = "mydataframe"):
        model = Prophet(interval_width=0.95,
                        growth='linear',
                        daily_seasonality=False,
                        weekly_seasonality=True,
                        yearly_seasonality=True,
                        seasonality_mode='multiplicative')
        model.fit(history_pd)
        future_pd = model.make_future_dataframe(periods=90, 
                                                freq='d', 
                                                include_history=True)
        forecast_pd = model.predict(future_pd)
        # types.mydataframe = bodo.typeof(sample_df)
        # print(forecast_pd.head())
        forecast_pd = forecast_pd[['ds', 'yhat']]
        
    return forecast_pd
        
forecast_pd = train_prophet(history_pd)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s][stdout:0] 
Initial log joint probability = -4.31318
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 250.565 1.05687e-07 81.5683 0.8065 0.8065 120 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 108 250.565 4.42979e-09 101.596 0.2056 0.2056 132 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
[stdout:6] 
Initial log joint probability = -3.75545
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 37 268.616 0.000121415 100.162 1.171e-06 0.001 89 LS failed, Hessian reset 
 96 268.633 9.82809e-08 103.767 9.583e-10 0.001 203 LS failed, Hessian reset 
 99 268.633 3.64772e-08 97.7594 1 1 207 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 102 268.633 9.96862e-09 98.5907 0.6173 0.6173 213 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
[stdout:3] 
Initial log joint probability = -4.85942
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 91 261.862 7.79413e-09 101.006 0.3291 0.3291 122 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
%px: 0%| | 0/12 [00:01<?, ?tasks/s][stdout:9] 
Initial log joint probability = -3.66336
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 79 255.851 9.94276e-09 98.705 0.6148 0.6148 103 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s][stdout:5] 
Initial log joint probability = -4.98065
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 80 260.358 6.79621e-09 103.159 0.4942 0.4942 113 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
[stdout:8] 
Initial log joint probability = -3.73117
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 97 281.808 8.90764e-09 95.9753 0.03827 0.3773 147 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
[stdout:2] 
Initial log joint probability = -4.80524
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 84 248.877 8.69177e-09 91.2901 0.3257 0.3257 111 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s][stdout:7] 
Initial log joint probability = -5.41741
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 265.033 3.43363e-07 96.5765 1 1 128 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 117 265.033 7.91591e-09 101.544 0.4699 0.4699 151 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
[stdout:10] 
Initial log joint probability = -4.88616
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 72 289.72 0.000308059 102.196 3.127e-06 0.001 144 LS failed, Hessian reset 
 99 289.779 1.17204e-05 94.5739 1 1 179 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 125 289.78 8.07232e-09 103.528 0.2469 0.2469 215 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
[stdout:4] 
Initial log joint probability = -4.16876
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 265.876 1.02634e-07 94.0756 1 1 129 
 Iter log prob ||dx|| ||grad|| alpha alpha0

In [0]:
%%px
forecast_pd.head(2)

Out[0:13]: 
 ds yhat
0 2013-01-01 10.263777
1 2013-01-02 11.175099
 Out[1:13]: 
 ds yhat
0 2013-06-09 31.783622
1 2013-06-10 19.513729
 Out[2:13]: 
 ds yhat
0 2013-11-21 20.494338
1 2013-11-22 21.421120
 Out[3:13]: 
 ds yhat
0 2014-04-26 21.077336
1 2014-04-27 23.593243
 Out[4:13]: 
 ds yhat
0 2014-09-25 16.227035
1 2014-09-26 18.947732
 Out[5:13]: 
 ds yhat
0 2015-03-03 15.216696
1 2015-03-04 14.580416
 Out[6:13]: 
 ds yhat
0 2015-08-04 21.542233
1 2015-08-05 22.786130
 Out[7:13]: 
 ds yhat
0 2016-01-10 18.375139
1 2016-01-11 9.348926
 Out[8:13]: 
 ds yhat
0 2016-06-09 25.310143
1 2016-06-10 26.531259
 Out[9:13]: 
 ds yhat
0 2016-11-17 22.896701
1 2016-11-18 22.476999
 Out[10:13]: 
 ds yhat
0 2017-04-20 22.248740
1 2017-04-21 24.541892
 Out[11:13]: 
 ds yhat
0 2017-09-26 22.406586
1 2017-09-27 22.159772

In [0]:
%%px

#file an issue for mean_squared_error, mean_absolute_error (it is supported but it is not working!!!)
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from datetime import date

@bodo.jit()
def evaluation_metrics(history_pd, forecast_pd):
    # get historical actuals & predictions for comparison
    forecast_pd['ds'] = pd.to_datetime(forecast_pd['ds']).dt.date
    history_pd['ds'] = pd.to_datetime(history_pd['ds']).dt.date
    actuals_pd = history_pd[ history_pd['ds'] < date(2018, 1, 1) ]['y']
    predicted_pd = forecast_pd[ forecast_pd['ds'] < pd.to_datetime('2018-01-01') ]['yhat']
    # calculate evaluation metrics
    with bodo.objmode():
        mae = mean_absolute_error(actuals_pd, predicted_pd)
        mse = mean_squared_error(actuals_pd, predicted_pd)
        rmse = sqrt(mse)
        # print metrics to the screen
        print( '\n'.join(['MAE: {0}', 'MSE: {1}', 'RMSE: {2}']).format(mae, mse, rmse) )
    

evaluation_metrics(history_pd, forecast_pd)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:03<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:04<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:05<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:06<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:07<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:08<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:09<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:10<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px: 0%| | 0/12 [00:11<?, ?tasks/s]%px:

[0:execute]: 
 --------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 /tmp/ipykernel_3253/752190812.py in <module> 
 20 
 21 
 ---> 22 evaluation_metrics ( history_pd , forecast_pd ) 

 /databricks/python/lib/python3.8/site-packages/sklearn/metrics/_regression.py in mean_absolute_error (y_true, y_pred, sample_weight, multioutput) 
 184 0.85 ... 
 185 """
 --> 186 y_type, y_true, y_pred, multioutput = _check_reg_targets(
 187 y_true , y_pred , multioutput
 188 )

 /databricks/python/lib/python3.8/site-packages/sklearn/metrics/_regression.py in _check_reg_targets (y_true, y_pred, multioutput, dtype) 
 87 the dtype argument passed to check_array . 
 88 """
 ---> 89 check_consistent_length ( y_true , y_pred ) 
 90 y_true = check_array ( y_true , ensure_2d = False , dtype = dtype ) 
 91 y_pred = check_array ( y_pred , ensure_2d = False , dtype = dtype ) 

 /databricks/python/lib/python3.8/site-packages/sklearn/utils/validation.py in check_consistent_length (*arrays) 
 329 uniques = np . unique ( lengths ) 
 330 if len ( uniques ) > 1 : 
 --> 331 raise ValueError(
 332 "Found input variables with inconsistent numbers of samples: %r" 
 333 % [ int ( l ) for l in lengths ] 

 ValueError : Found input variables with inconsistent numbers of samples: [159, 249]

[1:execute]: 
 --------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 /tmp/ipykernel_7568/752190812.py in <module> 
 20 
 21 
 ---> 22 evaluation_metrics ( history_pd , forecast_pd ) 

 /databricks/python/lib/python3.8/site-packages/sklearn/metrics/_regression.py in mean_absolute_error (y_true, y_pred, sample_weight, multioutput) 
 184 0.85 ... 
 185 """
 --> 186 y_type, y_true, y_pred, multioutput = _check_reg_targets(
 187 y_true , y_pred , multioutput
 188 )

 /databricks/python/lib/python3.8/site-packages/sklearn/metrics/_regression.py in _check_reg_targets (y_true, y_pred, multioutput, dtype) 
 87 the dtype argument passed to check_array . 
 88 """
 ---> 89 check_consistent_length ( y_true , y_pred ) 
 90 y_true = check_array ( y_true , ensure_2d = False , dtype = dtype ) 
 91 y_pred = check_array ( y_pred , ensure_2d = False , dtype = dtype ) 

 /databricks/python/lib/python3.8/site-packages/sklearn/utils/validation.py in check_consistent_length (*arrays) 
 329 uniques = np . unique ( lengths ) 
 330 if len ( uniques ) > 1 : 
 --> 331 raise ValueError(
 332 "Found input variables with inconsistent numbers of samples: %r" 
 333 % [ int ( l ) for l in lengths ] 

 ValueError : Found input variables with inconsistent numbers of samples: [165, 255]

[2:execute]: 
 --------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 /tmp/ipykernel_3220/752190812.py in <module> 
 20 
 21 
 ---> 22 evaluation_metrics ( history_pd , forecast_pd ) 

 /databricks/python/lib/python3.8/site-packages/sklearn/metrics/_regression.py in mean_absolute_error (y_true, y_pred, sample_weight, multioutput) 
 184 0.85 ... 
 185 """
 --> 186 y_type, y_true, y_pred, multioutput = _check_reg_targets(
 187 y_true , y_pred , multioutput
 188 )

 /databricks/python/lib/python3.8/site-packages/sklearn/metrics/_regression.py in _check_reg_targets (y_true, y_pred, multioutput, dtype) 
 87 the dtype argument passed to check_array . 
 88 """
 ---> 89 check_consistent_length ( y_true , y_pred ) 
 90 y_true = check_array ( y_true , ensure_2d = False , dtype = dtype ) 
 91 y_pred = check_array ( y_pred , ensure_2d = False , dtype = dtype ) 

 /databricks/python/lib/python3.8/site-packages/sklearn/utils/validation.py in check_consistent_length (*arrays) 
 329 uniques = np . unique ( lengths ) 
 330 if len ( uniques ) > 1 : 
 --> 331 raise ValueError(
 332 "Found input variables with inconsistent numbers of samples: %r" 
 333 % [ int ( l ) for l in lengths ] 

 ValueError : Found input variables with inconsistent numbers o

In [0]:
%%px

@bodo.jit
def agg_store_item(ip_df):

    grp_df = (ip_df
              .groupby(['store', 'item', 'date'], as_index=False)
              .agg({'sales': sum})
                )
    grp_df = grp_df.rename(columns = {'date' : 'ds', 'sales': 'y'})
    return grp_df

grp_df = agg_store_item(train)

if bodo.get_rank() == 0:
    display(grp_df.head(2))

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s][output:0]
 store item ds y
0 1 1 2013-01-15 5
1 1 1 2013-01-26 12
%px: 67%|██████▋ | 8/12 [00:01<00:00, 79.62tasks/s]%px: 100%|██████████| 12/12 [00:01<00:00, 9.13tasks/s]

In [0]:
%%px

time = pd.date_range('6/28/2013', periods=1, freq='1D')


sample_df = pd.DataFrame({"ds": [time[0]],
                          "y": [5623],
                          "yhat": [5623],
                          "yhat_upper": [5623],
                          "yhat_lower": [5623],
                          "store": [12],
                          "item": [12]}
                          )

from numba.core import types
types.mydataframe = bodo.typeof(sample_df)

In [0]:
%%px

@bodo.jit()
def train_prophet(history_pd):
    
    history_pd = history_pd.dropna()
    with bodo.objmode(results_pd = "mydataframe"):
        
        model = Prophet(interval_width=0.95,
                        growth='linear',
                        daily_seasonality=False,
                        weekly_seasonality=True,
                        yearly_seasonality=True,
                        seasonality_mode='multiplicative')

        # class suppress_stdout_stderr(object):

        #     """
        #     A context manager for doing a "deep suppression" of stdout and stderr in
        #     Python, i.e. will suppress all print, even if the print originates in a
        #     compiled C/Fortran sub-function.
        #     This will not suppress raised exceptions, since exceptions are printed
        #     to stderr just before a script exits, and after the context manager has
        #     exited (at least, I think that is why it lets exceptions through).
        #     """
            
        #     def __init__(self):
        #         # Open a pair of null files
        #         self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        #         # Save the actual stdout (1) and stderr (2) file descriptors.
        #         self.save_fds = [os.dup(1), os.dup(2)]

        #     def __enter__(self):
        #         # Assign the null pointers to stdout and stderr.
        #         os.dup2(self.null_fds[0], 1)
        #         os.dup2(self.null_fds[1], 2)

        #     def __exit__(self, *_):
        #         # Re-assign the real stdout/stderr back to (1) and (2)
        #         os.dup2(self.save_fds[0], 1)
        #         os.dup2(self.save_fds[1], 2)
        #         # Close the null files
        #         for fd in self.null_fds + self.save_fds:
        #             os.close(fd)
        
        # with suppress_stdout_stderr():
        #     model.fit(history_pd)

        model.fit(history_pd)

        future_pd = model.make_future_dataframe(periods=90, 
                                                freq='d', 
                                                include_history=True)
        forecast_pd = model.predict(future_pd)

        forecast_pd = forecast_pd[['ds', 'yhat', 'yhat_upper', 'yhat_lower']]

        # ASSEMBLE EXPECTED RESULT SET
        # --------------------------------------
        # get relevant fields from forecast
        f_pd = forecast_pd[['ds','yhat', 'yhat_upper', 'yhat_lower']].set_index('ds')
        
        # get relevant fields from history
        h_pd = history_pd[['ds','store','item','y']].set_index('ds')
        
        # join history and forecast
        results_pd = f_pd.join(h_pd, how='left')
        results_pd.reset_index(level=0, inplace=True)
        
        # get store & item from incoming data set
        results_pd['store'] = history_pd['store'].iloc[0]
        results_pd['item'] = history_pd['item'].iloc[0]

        
    return results_pd

In [0]:
%%px
import time

#@bodo.jit
def proph_fn(df):
    
    ip_df = df[(df['store'] == 1) & (df['item'] == 1)]
    #ip_df = df
    start_time = time.time()
    g = ip_df.groupby(['store', 'item'], as_index=False)
    fcst_df = g.apply(train_prophet)
    fcst_df.columns = ['ds', 'yhat', 'yhat_upper', 'yhat_lower', 'store', 'item','y']
    print("Execution time: ", time.time()-start_time, "seconds")
    
    fcst_df['training_date'] = pd.to_datetime('today')
    print(fcst_df.head(10))

    return fcst_df

In [0]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from datetime import date
from prophet import Prophet
import logging

from pyspark.sql.types import *

# structure of the training data set
train_schema = StructType([
  StructField('date', DateType()),
  StructField('store', IntegerType()),
  StructField('item', IntegerType()),
  StructField('sales', IntegerType())
  ])

# read the training file into a dataframe
train = spark.read.csv(
  '/mnt/rjml2/temp/demand-forecasting/train.csv', 
  header=True, 
  schema=train_schema
  )

# make the dataframe queriable as a temporary view
train.createOrReplaceTempView('train')

sql_statement = '''
  SELECT
    store,
    item,
    CAST(date as date) as ds,
    SUM(sales) as y
  FROM train
  WHERE store = 1 AND item = 1
  GROUP BY store, item, ds
  ORDER BY store, item, ds
  '''

store_item_history = (
  spark
    .sql( sql_statement )
    .repartition(sc.defaultParallelism, ['store', 'item'])
  ).cache()

from pyspark.sql.types import *

result_schema =StructType([
  StructField('ds',DateType()),
  StructField('store',IntegerType()),
  StructField('item',IntegerType()),
  StructField('y',FloatType()),
  StructField('yhat',FloatType()),
  StructField('yhat_upper',FloatType()),
  StructField('yhat_lower',FloatType())
  ])

def forecast_store_item( history_pd):
  
  # TRAIN MODEL AS BEFORE
  # --------------------------------------
  # remove missing values (more likely at day-store-item level)
  history_pd = history_pd.dropna()
  
  # configure the model
  model = Prophet(
    interval_width=0.95,
    growth='linear',
    daily_seasonality=False,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative'
    )
  
  # train the model
  model.fit( history_pd )
  # --------------------------------------
  
  # BUILD FORECAST AS BEFORE
  # --------------------------------------
  # make predictions
  future_pd = model.make_future_dataframe(
    periods=90, 
    freq='d', 
    include_history=True
    )
  forecast_pd = model.predict( future_pd )  
  # --------------------------------------
  
  # ASSEMBLE EXPECTED RESULT SET
  # --------------------------------------
  # get relevant fields from forecast
  f_pd = forecast_pd[ ['ds','yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')
  
  # get relevant fields from history
  h_pd = history_pd[['ds','store','item','y']].set_index('ds')
  
  # join history and forecast
  results_pd = f_pd.join( h_pd, how='left' )
  results_pd.reset_index(level=0, inplace=True)
  
  # get store & item from incoming data set
  results_pd['store'] = history_pd['store'].iloc[0]
  results_pd['item'] = history_pd['item'].iloc[0]
  # --------------------------------------
  
  # return expected dataset
  return results_pd[ ['ds', 'store', 'item', 'y', 'yhat', 'yhat_upper', 'yhat_lower'] ]  

from pyspark.sql.functions import current_date

results = (
  store_item_history
    .groupBy('store', 'item')
      .applyInPandas(forecast_store_item, schema=result_schema)
    .withColumn('training_date', current_date() )
    )

results.createOrReplaceTempView('new_forecasts')

display(results)

ds,store,item,y,yhat,yhat_upper,yhat_lower,training_date
2013-01-01,1,1,13.0,10.051273,18.253439,2.0496435,2021-10-14
2013-01-02,1,1,11.0,10.5286255,19.16495,2.4681573,2021-10-14
2013-01-03,1,1,14.0,11.053265,18.91158,2.4653366,2021-10-14
2013-01-04,1,1,13.0,12.244392,20.129028,3.7633965,2021-10-14
2013-01-05,1,1,10.0,13.780334,22.537075,5.3032236,2021-10-14
2013-01-06,1,1,12.0,14.37895,22.995806,5.5721154,2021-10-14
2013-01-07,1,1,10.0,7.8728924,17.192232,-1.0297664,2021-10-14
2013-01-08,1,1,9.0,9.901317,18.864975,1.6944209,2021-10-14
2013-01-09,1,1,12.0,10.315249,18.913128,2.0872517,2021-10-14
2013-01-10,1,1,9.0,10.787018,19.014244,2.263068,2021-10-14


In [0]:
%%px

new_forecasts = proph_fn(grp_df)

%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:00<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:01<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s][stdout:3] 
Initial log joint probability = -4.98036
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 245.88 4.84705e-05 89.6061 0.3286 1 128 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 118 245.905 8.18357e-05 97.5002 8.772e-07 0.001 186 LS failed, Hessian reset 
 165 245.912 9.48262e-08 85.8745 1.185e-09 0.001 281 LS failed, Hessian reset 
 173 245.912 4.57468e-09 72.9548 0.4491 0.4491 289 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
[stdout:0] 
Initial log joint probability = -15.1989
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 68 297.296 8.7284e-05 71.3325 1.697e-06 0.001 126 LS failed, Hessian reset 
 99 297.299 2.20391e-07 54.0719 1 1 180 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 115 297.299 1.91812e-08 62.1862 0.3179 1 199 
Optimization terminated normally: 
 Convergence detected: relative gradient magnitude is below tolerance
[stdout:6] 
Initial log joint probability = -4.36305
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 271.122 7.95331e-05 87.8049 0.6163 0.6163 133 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 199 271.193 1.19038e-07 64.2651 0.2867 1 266 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 299 271.198 5.05225e-07 74.4178 1 1 405 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 312 271.198 1.03554e-06 91.3937 1.413e-08 0.001 466 LS failed, Hessian reset 
 336 271.199 1.18793e-08 64.5707 0.3265 1 497 
Optimization terminated normally: 
 Convergence detected: relative gradient magnitude is below tolerance
%px: 0%| | 0/12 [00:02<?, ?tasks/s][stdout:9] 
Initial log joint probability = -5.53794
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 275.993 0.000813535 90.273 7.783e-06 0.001 177 LS failed, Hessian reset 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 199 276.03 2.14756e-06 88.4152 0.3108 0.3108 311 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 239 276.031 1.7449e-06 99.3825 2.025e-08 0.001 400 LS failed, Hessian reset 
 252 276.031 2.35211e-08 67.2042 0.2367 1 423 
Optimization terminated normally: 
 Convergence detected: relative gradient magnitude is below tolerance
%px: 0%| | 0/12 [00:02<?, ?tasks/s]%px: 0%| | 0/12 [00:02<?, ?tasks/s][stdout:11] 
Initial log joint probability = -3.51664
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 285.77 1.00838e-05 99.5819 1 1 119 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 128 285.771 6.67289e-09 89.982 0.2932 0.2932 156 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was below tolerance
%px: 0%| | 0/12 [00:02<?, ?tasks/s][stdout:5] 
Initial log joint probability = -6.69035
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 99 269.704 6.42148e-07 101.114 0.7351 0.7351 128 
 Iter log prob ||dx|| ||grad|| alpha alpha0 # evals Notes 
 114 269.704 6.23565e-07 102.188 6.242e-09 0.001 194 LS failed, Hessian reset 
 149 269.704 9.50397e-09 99.769 0.4294 0.4294 239 
Optimization terminated normally: 
 Convergence detected: absolute parameter change was bel

In [0]:
%%px

print(new_forecasts.head())

[stdout:1] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-01-07 7.248399 13.915102 -0.029153 1 1 10.0 
 1 2013-01-09 9.588004 16.406175 2.376506 1 1 12.0 
 2 2013-01-11 11.259292 18.579625 3.988343 1 1 9.0 
 3 2013-01-18 10.682747 17.299286 3.684469 1 1 7.0 
 4 2013-01-20 13.666064 21.222391 7.015925 1 1 15.0 

 training_date 
0 0 2021-10-14 16:47:37.136613 
 1 2021-10-14 16:47:37.136613 
 2 2021-10-14 16:47:37.136613 
 3 2021-10-14 16:47:37.136613 
 4 2021-10-14 16:47:37.136613 
[stdout:7] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-01-05 12.552865 19.972146 4.840005 1 1 10.0 
 1 2013-01-13 13.907954 22.400118 6.161226 1 1 10.0 
 2 2013-01-19 14.906205 22.508868 7.297163 1 1 18.0 
 3 2013-02-03 17.127085 25.752018 9.216338 1 1 15.0 
 4 2013-02-09 16.190490 23.893320 8.704867 1 1 14.0 

 training_date 
0 0 2021-10-14 16:47:36.691427 
 1 2021-10-14 16:47:36.691427 
 2 2021-10-14 16:47:36.691427 
 3 2021-10-14 16:47:36.691427 
 4 2021-10-14 16:47:36.691427 
[stdout:4] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-02-01 11.647520 19.171533 3.837993 1 1 11.0 
 1 2013-02-24 15.201081 23.188646 7.325343 1 1 11.0 
 2 2013-02-25 7.134731 14.771019 -0.160340 1 1 7.0 
 3 2013-03-04 9.695911 17.791632 1.510012 1 1 14.0 
 4 2013-03-08 16.581680 24.803188 9.635293 1 1 15.0 

 training_date 
0 0 2021-10-14 16:47:36.881911 
 1 2021-10-14 16:47:36.881911 
 2 2021-10-14 16:47:36.881911 
 3 2021-10-14 16:47:36.881911 
 4 2021-10-14 16:47:36.881911 
[stdout:10] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-01-03 10.550309 18.108540 2.894740 1 1 14.0 
 1 2013-01-12 12.354452 20.107152 4.402385 1 1 7.0 
 2 2013-01-22 7.886078 15.602457 0.389126 1 1 7.0 
 3 2013-01-25 12.148358 19.960506 4.932871 1 1 14.0 
 4 2013-01-31 12.435346 19.958227 4.419079 1 1 13.0 

 training_date 
0 0 2021-10-14 16:47:36.787275 
 1 2021-10-14 16:47:36.787275 
 2 2021-10-14 16:47:36.787275 
 3 2021-10-14 16:47:36.787275 
 4 2021-10-14 16:47:36.787275 
[stdout:3] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-01-04 15.509957 23.863034 7.267456 1 1 13.0 
 1 2013-01-08 11.460843 20.092722 3.320193 1 1 9.0 
 2 2013-01-29 7.008015 15.041799 -1.112369 1 1 6.0 
 3 2013-02-08 13.105923 21.768215 4.265969 1 1 11.0 
 4 2013-02-28 11.854286 20.382127 3.616386 1 1 10.0 

 training_date 
0 0 2021-10-14 16:47:35.668558 
 1 2021-10-14 16:47:35.668558 
 2 2021-10-14 16:47:35.668558 
 3 2021-10-14 16:47:35.668558 
 4 2021-10-14 16:47:35.668558 
[stdout:0] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-01-15 9.467182 17.194721 0.793884 1 1 5.0 
 1 2013-01-26 13.255736 21.925081 4.196942 1 1 12.0 
 2 2013-01-27 16.604350 25.485621 8.081494 1 1 12.0 
 3 2013-02-05 10.495106 19.162753 1.580495 1 1 9.0 
 4 2013-02-06 11.179522 19.591978 2.580602 1 1 10.0 

 training_date 
0 0 2021-10-14 16:47:35.646700 
 1 2021-10-14 16:47:35.646700 
 2 2021-10-14 16:47:35.646700 
 3 2021-10-14 16:47:35.646700 
 4 2021-10-14 16:47:35.646700 
[stdout:9] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-01-10 12.257574 19.731502 4.947933 1 1 9.0 
 1 2013-01-17 10.878963 18.544439 3.737704 1 1 16.0 
 2 2013-01-21 7.264897 14.822748 -0.034612 1 1 8.0 
 3 2013-01-23 9.627350 17.561008 2.438558 1 1 9.0 
 4 2013-02-19 12.259293 19.787766 4.291022 1 1 10.0 

 training_date 
0 0 2021-10-14 16:47:35.474833 
 1 2021-10-14 16:47:35.474833 
 2 2021-10-14 16:47:35.474833 
 3 2021-10-14 16:47:35.474833 
 4 2021-10-14 16:47:35.474833 
[stdout:6] 
 ds yhat yhat_upper yhat_lower store item y \
0 0 2013-01-01 12.704121 20.154747 5.360770 1 1 13.0 
 1 2013-01-24 11.935379 19.746255 4.622425 1 1 8.0 
 2 2013-01-28 8.128158 15.659876 -0.199125 1 1 11.0 
 3 2013-02-15 13.869827 20.971717 6.131659 1 1 11.0 
 4 2013-02-23 13.911128 21.970609 6.464884 1 1 15.0 

 training_date 
0 0 2021-10-14 16:47:35.425200 
 1 2021-10-14 16:47:35.425200 
 2 2021-10-14 16:47:35.425200 
 3 2021-10-14 16:47:35.425200 
 4 2021-10-14 16:47:35.425200 
[stdout:5] 
 ds yhat yhat_upper yhat_lowe

In [0]:
%%px

#@bodo.jit()
def save_data(new_forecasts):
    forecast_tosave = new_forecasts.rename(columns = {"ds" : "date", "y" : "sales", "yhat" : "sales_predicted",
                                                    "yhat_upper" : "sales_predicted_upper",
                                                    "yhat_lower" : "sales_predicted_lower"})

    forecast_tosave.to_parquet("/dbfs/mnt/temprj2/data/demand-forecasting/forecasts.pq")


save_data(new_forecasts)

In [0]:
%%px
# define function to calculate metrics

@bodo.jit()
def evaluate_forecast(evaluation_pd):
  
  # calulate evaluation metrics
    mae = mean_absolute_error( evaluation_pd['y'], evaluation_pd['yhat'] )
    mse = mean_squared_error( evaluation_pd['y'], evaluation_pd['yhat'] )
    rmse = sqrt( mse )
    
    # get store & item in incoming data set
    training_date = evaluation_pd['training_date'].iloc[0]
    
    store = evaluation_pd['store'].iloc[0]
    item = evaluation_pd['item'].iloc[0]
  
  # assemble result set
    results = {'training_date':[training_date], 'store':[store], 'item':[item], 'mae':[mae], 'mse':[mse], 'rmse':[rmse]}
    results_df = pd.DataFrame( results )
    #print(results_df)
    return results_df

In [0]:
%%px

#@bodo.jit()
def clean_data(df):
    df = df.dropna()
    return df

new_forecasts = clean_data(new_forecasts)

In [0]:
%%px

#@bodo.jit()
def calculate_metrics(df):
    df = df.loc[df['ds'] < '2018-01-01']
    df = df[['training_date', 'store', 'item', 'y', 'yhat']]
    new_forecast_evals = df.groupby(['training_date', 'store', 'item'], as_index = False).apply(evaluate_forecast)
    print(new_forecast_evals)
    
calculate_metrics(new_forecasts)